In [1]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import *
import matplotlib.colors as mcolors
from skimage import filters, segmentation, measure, morphology, feature
from scipy import ndimage as ndi

In [ ]:
def intensity_median(regionmask, intensity):
    return np.median(intensity[regionmask])

def intensity_std(regionmask, intensity):
    return np.std(intensity[regionmask])

def gradient_max_intensity(regionmask, intensity):
    edges = filters.sobel(intensity * regionmask)
    idx_max = np.unravel_index(np.argmax(intensity * regionmask), intensity.shape)
    return edges[idx_max]

def mean_gradient(regionmask, intensity):
    edges = filters.sobel(intensity)
    return np.mean(edges[regionmask])

def std_gradient(regionmask, intensity):
    edges = filters.sobel(intensity)
    return np.std(edges[regionmask])

In [ ]:
df = pd.DataFrame(columns=['Sample ID', 'Rock ID', 'Class', 'Orientation', 'Perimeter', 'Area', 'Axis Major Length', 'Axis Minor Length', 'Mean Height', 'Median Height', 'Max Height', 'STD Height', 'Mean Gradient', 'STD Gradient', 'Gradient at Max', 'Weighted Hu Moment 1', 'Weighted Hu Moment 2', 'Weighted Hu Moment 3', 'Weighted Hu Moment 4'])

n_samples = 4
classes = ['12','58', '34']

for idx_cls, cls in enumerate(classes):
    fig, axs = subplots(nrows=4, ncols=n_samples, figsize=(12,10), tight_layout=True)
    for idx_samp in range(n_samples):
        scan = np.load(f'../data/{cls}_sample{idx_samp+1:d}.npy')
        s1_scan = filters.gaussian(scan, sigma=3) # smoothing
        s2_scan = filters.median(s1_scan, footprint=np.ones((3,)*2)) # smoothing
        
        binary = scan > filters.threshold_li(s2_scan)
        s1_binary = morphology.binary_opening(binary, morphology.disk(3))
        s2_binary = morphology.binary_dilation(s1_binary)
        
        rocks = (scan * s2_binary)
        distance = ndi.distance_transform_edt(rocks)
        coords = feature.peak_local_max(distance, min_distance=8)
        mask = np.zeros_like(distance, dtype=bool)
        mask[tuple(coords.T)] = True
        markers, _ = ndi.label(mask)
        labels = segmentation.watershed(-distance, markers, mask=s2_binary)
        
        dilated_mask = morphology.binary_dilation(mask, footprint=np.ones((10,)*2))
        cmap = cm.get_cmap('tab10')(np.linspace(0, 1, len(np.unique(labels))))
        cmap = np.random.permutation(cmap)
        cmap[0,:-1] = 0
        cmap = mcolors.ListedColormap(cmap)
        
        axs[0,idx_samp].imshow(scan, cmap="hot")
        
        axs[1,idx_samp].imshow(scan * (1-s2_binary), cmap="hot")

        axs[2,idx_samp].imshow(rocks, cmap='hot', alpha=0.5)
        axs[2,idx_samp].imshow(np.ma.masked_where(dilated_mask == 0, dilated_mask))
        
        axs[3,idx_samp].imshow(labels, cmap=cmap)
        
        for idx_ax in range(4):
            axs[idx_ax,idx_samp].set_axis_off()
            axs[idx_ax,idx_samp].set_aspect('equal')

        regions = measure.regionprops(labels, scan, extra_properties=(intensity_median, intensity_std, gradient_max_intensity,mean_gradient,std_gradient))
        
        for idx_reg, region in enumerate(regions):
            df.loc[len(df.index)] = [idx_samp+1, idx_reg+1, cls, region.orientation, region.perimeter, region.area, region.axis_major_length, region.axis_minor_length, region.intensity_mean, region.intensity_median, region.intensity_max, region.intensity_std, region.mean_gradient, region.std_gradient, region.gradient_max_intensity, *region.moments_weighted_hu[:4]]
    show()

In [ ]:
fig, axs = subplots(nrows=n_samples, ncols=len(classes), figsize=(10,10), tight_layout=True)

for idx_cls, cls in enumerate(classes):
    for idx_samp in range(n_samples):
        scan = np.load(f'../data/{cls}_sample{idx_samp+1:d}.npy')
        s1_scan = filters.gaussian(scan, sigma=3) # smoothing
        s2_scan = filters.median(s1_scan, footprint=np.ones((3,)*2)) # smoothing
        
        binary = scan > filters.threshold_li(s2_scan)
        s1_binary = morphology.binary_opening(binary, morphology.disk(3))
        s2_binary = morphology.binary_dilation(s1_binary)
        
        rocks = (scan * s2_binary)
        distance = ndi.distance_transform_edt(rocks)
        coords = feature.peak_local_max(distance, min_distance=8)
        mask = np.zeros_like(distance, dtype=bool)
        mask[tuple(coords.T)] = True
        markers, _ = ndi.label(mask)
        labels = segmentation.watershed(-distance, markers, mask=s2_binary)
        
        dilated_mask = morphology.binary_dilation(mask, footprint=np.ones((10,)*2))
        
        axs[idx_samp,idx_cls].imshow(rocks, cmap='hot', alpha=0.5)
        axs[idx_samp,idx_cls].imshow(np.ma.masked_where(dilated_mask == 0, dilated_mask))
        axs[idx_samp,idx_cls].set_aspect('equal')
        axs[idx_samp,idx_cls].set_axis_off()
        
        print(f'class={cls}, samp_id={idx_samp}: #est_rocks = {(mask).sum()}')
        
show()

In [ ]:
fig, axs = subplots(nrows=n_samples, ncols=len(classes), figsize=(10,10), tight_layout=True)

for idx_cls, cls in enumerate(classes):
    for idx_samp in range(n_samples):
        scan = np.load(f'../data/{cls}_sample{idx_samp+1:d}.npy')
        s1_scan = filters.gaussian(scan, sigma=3) # smoothing
        s2_scan = filters.median(s1_scan, footprint=np.ones((3,)*2)) # smoothing
        
        binary = scan > filters.threshold_li(s2_scan)
        s1_binary = morphology.binary_opening(binary, morphology.disk(3))
        s2_binary = morphology.binary_dilation(s1_binary)
        
        rocks = (scan * s2_binary)
        distance = ndi.distance_transform_edt(rocks)
        coords = feature.peak_local_max(distance, min_distance=8)
        mask = np.zeros_like(distance, dtype=bool)
        mask[tuple(coords.T)] = True
        markers, _ = ndi.label(mask)
        labels = segmentation.watershed(-distance, markers, mask=s2_binary)

        cmap = cm.get_cmap('tab10')(np.linspace(0, 1, len(np.unique(labels))))
        cmap = np.random.permutation(cmap)
        cmap[0,:-1] = 0
        cmap = mcolors.ListedColormap(cmap)
        
        axs[idx_samp,idx_cls].imshow(labels, cmap=cmap)
        
        axs[idx_samp,idx_cls].set_aspect('equal')
        axs[idx_samp,idx_cls].set_axis_off()
show()

In [ ]:
df.to_csv('../data/rock_data.csv', index=False)